# Value Lost

A notebook to determine the value lost when applying a maintenance strategy defined in an asset model to the population.

Requested: greg.bell@essentialenergy.com.au
Author: gavin.treseder@essentialenergy.com.au

## Project Setup

In [1]:
import copy
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from IPython.display import clear_output
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as ss

from pof.system import System
from pof.interface.figures import make_sensitivity_fig, make_value_lost_fig, make_contour_plot
from pof.loader.asset_model_loader import AssetModelLoader
from pof.paths import Paths
from pof.units import scale_units_val

%matplotlib inline

# Load Data
## Load the Asset Model
The asset model needs to be loaded from the excel file and converted into a component object that execute simulations

In [2]:
asset_model_filename = "Asset Model.xlsx"

paths = Paths()
model_path = paths.demo_path + os.sep + asset_model_filename

try:
    aml = AssetModelLoader()
    pof_data = aml.load(model_path)
    system = System.from_dict(pof_data['overhead_network'])
    comp = system.comp['pole']
    print("Asset Model loaded")
    pass
except:
    comp = Component.demo()
    print("Demo Model loaded")

Asset Model loaded


In [3]:
def rolling_mean(df, x_var:str, y_var:str, window=5):
    """ Returns a df with a rolling mean"""
    #TODO set this up so it work for
    #TODO move this to expected_cost

    df_rolling = (
        df.copy(deep=True)
        .rename(columns={'task':'source'})
        .dropna()
        .sort_values(by=['source', x_axis, y_var])
    )

    # Reduce the window size if not enough values are passed
    window = max(0,min(len(df_rolling[y_var].unique()) - 1, window))

    # Generate a rolling average
    df_rolling = df_rolling.groupby(by=['source', y_axis]).rolling(5).mean().drop([y_var], axis=1).reset_index().dropna()

    df_rolling['active'] = df_rolling['active'].astype(bool)

    return df_rolling


In [4]:
def calc_value_lost(df, x_var:str, y_var:str, z_var='cost_cumulative'):
    """ Takes a df of risk costs and returns a df with teh value lost
    # TODO move this to risk cost
    # TODO check y_var and x_var are the right way around
    """

    prefix = ["quantity", "cost"]
    suffix = ["_cumulative", '_lifecycle']
    cols = [f"{pre}{suf}" for pre in prefix for suf in suffix]

    # Calculate the total cost for each scenario
    df = df_rolling.groupby(by=[x_var, y_var])[cols].sum()

    # Calculate the value_lost at each interval
    df['value_lost'] = df[z_var] - df.groupby(by=[y_var])[z_var].transform('min') 

    return df.reset_index()

## Long run simulation 
Run a long simulation to repeat the process above for the key outputs without the interim steps

In [5]:
# Set the parameters in the model_units
model_units = 'months'
t_end = 1200
n_iterations = 500
sens_vars = {
    'pole-consequence-cost': np.concatenate([np.arange(2e4, 1e5,2e4), np.arange(1e5,6e5,1e5)]),
    'pole-task_group_name-groundline-t_interval': np.arange(12, 120, 6)
}

x_axis = 'pole-task_group_name-groundline-t_interval'
y_axis = 'pole-consequence-cost'
z_axis = 'value_lost'

window=5

In [6]:
# Run the simulation
comp.units = model_units

df_sens_chain = comp.sensitivty_chain(sens_vars, t_end=t_end, n_iterations=n_iterations)

In [38]:
# Get ready for plotting
df_rolling = rolling_mean(df=df_sens_chain, x_var=x_axis, y_var=y_axis, window=window)

df_total = calc_value_lost(df=df_rolling, x_var=x_axis, y_var=y_axis, z_var='cost_lifecycle')

In [39]:
make_contour_plot(df_total, x_axis=x_axis, y_axis=y_axis, z_axis=z_axis)

In [40]:
px.line(df_total.reset_index(), x=x_axis, y=z_axis, color=y_axis)

In [131]:
df_total.to_csv(paths.output_path + os.sep + "interim_model_results_200804.csv")

# Match Asset Models to the population
## Load the population data

In [24]:
filename = "ACS - Poles - Consequence Model Output.csv"
print("File Loading...")
df_cons = pd.read_csv(paths.csv_path + os.sep + filename)
clear_output()
df_cons.head()

,Unnamed: 0,ASSET_ID,Depot (ZSG Area),Pole_LU_NSWMajo,Pole_LU_NSWDeta,Pole_LU_ALUMMaj,Pole_LU_ALUMDet,Road Crossing Conductor Type,HV_LV Status,BushfirePriority,...,C_Environment_Rank,Site_Bushfire_Moderate_Probability,Site_Bushfire_Severe_Probability,C_Bushfire_Dollars,Site_Environment_Insignificant_Probability,Site_Environment_Minor_Probability,C_Environment_Dollars,C_Financial_Dollars,Total Consequence $,Radial(Y/N)
0,0,388,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2802.19134,21198.144867,NaN
1,1,389,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2802.57978,21198.533307,NaN
2,2,390,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2803.09692,21199.050447,NaN
3,3,391,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2803.76382,21199.717347,NaN
4,4,392,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2804.33010,21200.283627,NaN


## Prepare the population Data

In [25]:
# TODO turn ino a function so it works for mutliple y vars
pop_var = 'Total Consequence $'
sens_var = 'pole-consequence-cost'
uid_col = 'ASSET_ID'
agg_cols = ['Depot (ZSG Area)']

df_sens_var = pd.DataFrame(sens_vars[sens_var], columns= [sens_var])

# Fill with average val
df_cons[pop_var] = df_cons[pop_var].fillna(df_cons[pop_var].mean())

# Match to the closest value from the model
df = pd.merge_asof(
    df_cons.sort_values(pop_var),
    df_sens_var,
    left_on = pop_var,
    right_on = sens_var,
    direction = 'forward'
)

# Remove columsns that aren't required
col_to_keep = [uid_col] + agg_cols + [pop_var] + [sens_var]
df = df[col_to_keep]

# Count into sub populations
by = agg_cols + [sens_var]
df_pop = df.groupby(by=by)[uid_col].count().reset_index().rename(columns={uid_col:'asset_count'})
df_pop.head()

,Depot (ZSG Area),pole-consequence-cost,asset_count
0,Albury Depot,20000.0,7506
1,Albury Depot,40000.0,5195
2,Albury Depot,60000.0,113
3,Albury Depot,80000.0,385
4,Albury Depot,100000.0,919


## Combine the models together

In [26]:
#Combine the two files together
df_result = pd.merge(df_pop, df_total, on=sens_var)

# Calculte the value lost across the population
df_result['value_lost_pop'] = df_result['value_lost'] * df_result['asset_count']

# Caculate the value lost across ignoring sub populations
df_all = df_result.groupby(by=x_axis).sum()[['value_lost', 'value_lost_pop']].reset_index()
df_all[agg_cols] = "all"

# Calculate the value lost within sub populations
by = [x_axis] + agg_cols
df_agg = df_result.groupby(by=by).sum()[['value_lost', 'value_lost_pop']].reset_index()

# Combine them together
df_all = df_all.append(df_agg)

# Plot the results
make_value_lost_fig(df_all, x_axis=x_axis, y_axis='value_lost_pop', legend =agg_cols[0], title=f'Annual Value Lost')

## 1 step version, which seems causes memory errors

In [ ]:
# df_sens_var = pd.DataFrame(sens_vars[y_var], columns= [y_var])

# # Fill with average val
# df_cons[pop_var] = df_cons[pop_var].fillna(df_cons[pop_var].mean())

# # Match to the closest value from the model
# df = pd.merge_asof(
#     df_cons.sort_values(pop_var),
#     df_sens_var,
#     left_on = pop_var,
#     right_on = y_var,
#     direction = 'forward'
# )

# df_plot = pd.merge(df, df_total.reset_index(), on=y_var).groupby(by=x_var).sum().reset_index()
# #y= ['list(set(df_plot) - set([sens_var, uid_col]))']
# y = ['quantity_annual', 'cost_annual', 'value_lost']

# px.line(df_plot, x=x_var, y=y)